# LDA - GBT Pipeline 

## Import necessary packages

In [1]:
import time
import optuna
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

from dataset.dataset import Dataset
from constants import CLEANED_DATASET_PATH

## Constants

In [5]:
RANDOM_SEED = 0

## Load the Hate Speech Filipino dataset from Hugging Face

In [6]:
dataset = Dataset()
X, Y = dataset.load_from_file(CLEANED_DATASET_PATH)

Data loaded from dataset/cleaned_dataset.pkl


In [7]:
# Re-split the dataset into training, validation, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=RANDOM_SEED)

## Vectorize the texts to be able to perform LDA

In [8]:
# Define the CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2)
X_train_vector = vectorizer.fit_transform(X_train)
X_val_counts = vectorizer.transform(X_val)
X_test_counts = vectorizer.transform(X_test)

## Perform Latent Dirichlet Allocation on the training set

In [6]:
N_TOPICS = 10
print(f"Performing Latent Dirichlet Allocation for {N_TOPICS} topics")
lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = lda.fit_transform(X_train_vector)
X_val_topics = lda.transform(X_val_counts)
X_test_topics = lda.transform(X_test_counts)
print(f"Done performing Latent Dirichlet Allocation for {N_TOPICS} topics")

Performing Latent Dirichlet Allocation for 10 topics
Done performing Latent Dirichlet Allocation for 10 topics


## Train the Gradient-boosting Tree

In [7]:
# Define the objective function for Optuna
def create_objective(X_train, Y_train, X_test, Y_test):
    def objective(trial):
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)
        learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2)
        max_depth = trial.suggest_int("max_depth", 10, 100)
        gbt = GradientBoostingClassifier(n_estimators=n_estimators, 
                                         learning_rate=learning_rate, 
                                         max_depth=max_depth, 
                                         random_state=RANDOM_SEED)
        gbt.fit(X_train, Y_train)
        Y_pred = gbt.predict(X_test)
        score = accuracy_score(Y_test, Y_pred)
        return score
    return objective

In [8]:
# Create a study object
study = optuna.create_study(direction="maximize", study_name=f"LDA_GBT_Pipeline")

# Create the study objective
objective = create_objective(X_train_topics, Y_train, X_val_topics, Y_val)

# Execute an optimization
study.optimize(objective, n_trials=20, n_jobs=-1)

# Print the best trial results
print(f"Best Accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2024-04-30 22:00:10,196] A new study created in memory with name: LDA_GBT_Pipeline
[I 2024-04-30 22:02:37,125] Trial 6 finished with value: 0.6294691224268689 and parameters: {'n_estimators': 186, 'learning_rate': 0.003427667998654015, 'max_depth': 15}. Best is trial 6 with value: 0.6294691224268689.
[I 2024-04-30 22:06:16,746] Trial 5 finished with value: 0.6294691224268689 and parameters: {'n_estimators': 311, 'learning_rate': 0.008051735631311624, 'max_depth': 17}. Best is trial 6 with value: 0.6294691224268689.
[I 2024-04-30 22:07:04,764] Trial 7 finished with value: 0.6370530877573131 and parameters: {'n_estimators': 697, 'learning_rate': 0.00112363875879991, 'max_depth': 13}. Best is trial 7 with value: 0.6370530877573131.
[I 2024-04-30 22:07:59,430] Trial 8 finished with value: 0.605092091007584 and parameters: {'n_estimators': 295, 'learning_rate': 0.0013649199492631896, 'max_depth': 25}. Best is trial 7 with value: 0.6370530877573131.


In [ ]:
best_n_estimators = study.best_trial.params["n_estimators"]
best_learning_rate = study.best_trial.params["learning_rate"]
best_max_depth = study.best_trial.params["max_depth"]

gbt = GradientBoostingClassifier(n_estimators=best_n_estimators, 
                                 learning_rate=best_learning_rate,
                                 max_depth=best_max_depth,
                                 random_state=RANDOM_SEED)
gbt.fit(X_train_topics, Y_train)
Y_pred = gbt.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
print(f"Best LDA-GBT Model Accuracy: {score}")

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# 
# tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
# X_train_vector = tfidf_vectorizer.fit_transform(X_train)

In [9]:
from sklearn.decomposition import NMF

N_TOPICS = 20
print(f"Performing Non-negative Matrix Factorization for {N_TOPICS} topics")
nmf = NMF(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = nmf.fit_transform(X_train_vector)
X_val_topics = nmf.transform(X_val_counts)
X_test_topics = nmf.transform(X_test_counts)
print(f"Done performing Non-negative Matrix Factorization for {N_TOPICS} topics")

Performing Non-negative Matrix Factorization for 20 topics
Done performing Non-negative Matrix Factorization for 20 topics


In [11]:
gbt = GradientBoostingClassifier(n_estimators=697, 
                                 learning_rate=0.00112363875879991,
                                 max_depth=13,
                                 random_state=RANDOM_SEED)
gbt.fit(X_train_topics, Y_train)
Y_pred = gbt.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
print(f"Best NMF-GBT Model Accuracy: {score}")

Best NMF-GBT Model Accuracy: 0.6497022198159177
